# Лекция 1: Въведение в машинното самообучение за NLP

## От проблем до решение: Основи на ML с текстови данни

**Продължителност:** 2-2.5 часа  
**Предпоставки:** Основи на Python, линейна алгебра, вероятности  
**Следваща лекция:** Езикови модели и представяне на думи

---
## Цели на лекцията

След тази лекция ще можете:

- Формулирате NLP задача като ML проблем (вход, изход, хипотеза)
- Разбирате разликата между обучение с учител, без учител и с подсилване
- Идентифицирате пренагаждане (overfitting) и ъндърфитване в текстови модели
- Избирате подходящи метрики за оценка на NLP системи
- Прилагате основния ML workflow за текстова класификация

### Пътна карта за днес

```
Мотивация → Формулиране → Терминология → Парадигми → Генерализация → Метрики → Следващи стъпки
```

In [ ]:
# Импортиране на необходимите библиотеки
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

In [ ]:
# Scikit-learn за ML
from sklearn.datasets import fetch_20newsgroups
from sklearn.model_selection import train_test_split, learning_curve
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)

In [ ]:
# Настройки за визуализации
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

print("✓ Всички библиотеки са заредени успешно!")

---
## 1. Мотивация: Защо ML за език?

### Какво е изкуствен интелект, машинно самообучение и NLP?

**Изкуствен интелект (AI):** Широка област, която се занимава със създаването на интелигентни машини.

**Машинно самообучение (ML):** Подход в AI, при който системите се учат от данни, вместо да бъдат програмирани изрично.

**Обработка на естествен език (NLP):** Приложение на ML за разбиране и генериране на човешки език.

### Защо езикът е труден за компютрите?

**Двусмисленост (Ambiguity)**
- "Видях човек с телескоп" - Кой има телескопа?
- "Банката е затворена" - Финансова институция или речен бряг?

**Контекст**
- "Страхотно" може да е положително ("Страхотен филм!") или саркастично ("Страхотно, закъснях пак...")

**Вариативност**
- Едно и също значение, много начини да го кажем
- "Филмът ми хареса" ≈ "Страхотен филм" ≈ "10/10, препоръчвам"

In [ ]:
# Демонстрация: Едни и същи думи, различно значение
examples = [
    ("Този филм е невероятен!", "положително"),
    ("Невероятно, че пуснаха такъв филм.", "отрицателно"),
    ("Актьорите играят добре.", "положително"),
    ("Е, добре де... поне свърши.", "отрицателно"),
]

print("🎭 Примери за двусмисленост в sentiment analysis:\n")
for text, sentiment in examples:
    emoji = "😊" if sentiment == "положително" else "😞"
    print(f"{emoji} [{sentiment:12s}] \"{text}\"")

print("\n💡 Наблюдение: Думи като 'невероятен' и 'добре' могат да са и положителни, и отрицателни!")

### Успехи на NLP днес

**Машинен превод**
- Google Translate, DeepL
- Превод на документи в реално време

**Разговорни асистенти**
- ChatGPT, Claude, Gemini
- Отговарят на въпроси, пишат код, генерират текст

**Бизнес приложения**
- Sentiment analysis за анализ на клиентски отзиви
- Автоматична категоризация на имейли
- Извличане на информация от документи

---
## 2. Формулиране на ML проблем

### Какво означава "машина се учи"?

**Дефиниция (Tom Mitchell, 1997):**

> Компютърна програма се учи от опит **E** по отношение на задача **T** и метрика **P**, ако нейната производителност по **T**, измерена с **P**, се подобрява с опита **E**.

**Пример: Spam филтър**
- **T (Задача):** Класифицирай имейл като spam или не-spam
- **E (Опит):** База от етикетирани имейли
- **P (Метрика):** Процент правилно класифицирани имейли

### Формална нотация

**Входно пространство** $\mathcal{X}$: Множество от всички възможни входове (текстове, изречения, документи)

**Изходно пространство** $\mathcal{Y}$: Множество от възможни изходи (етикети, категории, преводи)

**Тренировъчни примери:** $(x^{(1)}, y^{(1)}), (x^{(2)}, y^{(2)}), \ldots, (x^{(n)}, y^{(n)})$

**Хипотеза (модел):** $h: \mathcal{X} \to \mathcal{Y}$

**Цел:** Намери хипотеза $h$, която добре предсказва $y$ за нови $x$.

In [ ]:
# Конкретен пример: Spam класификация
spam_examples = [
    ("Честито! Спечелихте $1,000,000! Кликнете тук!", "spam"),
    ("Здравей, утре ще се видим за кафе?", "not spam"),
    ("БЕЗПЛАТНО!!! Изтеглете сега!!!", "spam"),
    ("Изпращам ти файловете от срещата.", "not spam"),
    ("Купи Виагра на ниски цени!", "spam"),
    ("Потвърждавам участие в конференцията.", "not spam"),
]

print("📧 Тренировъчни данни за spam класификация:\n")
print(f"{'Текст':<50} {'Етикет':>10}")
print("=" * 62)

for text, label in spam_examples:
    short_text = text[:47] + "..." if len(text) > 50 else text
    print(f"{short_text:<50} {label:>10}")

print(f"\n💡 Това е X (входове) и Y (етикети). Моделът трябва да научи връзката!")

---
## 3. Основна ML терминология с текстови данни

### Признаци (Features) в NLP

Компютрите работят с числа, не с текст. Трябва да **превърнем текста в числа**.

**Подходи за представяне на текст:**
- **Bag-of-Words (BoW):** Брои появявания на думи
- **TF-IDF:** Претеглено броене (рядките думи тежат повече)
- **N-грами:** Брои поредици от N думи
- **Embeddings:** Плътни вектори (Лекция 2)

In [ ]:
# Демонстрация: Bag-of-Words
texts = [
    "Обичам този филм",
    "Мразя този филм",
    "Филмът е страхотен"
]

# Създаваме BoW представяне
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(texts)

# Показваме резултата
feature_names = vectorizer.get_feature_names_out()
bow_df = pd.DataFrame(X.toarray(), columns=feature_names, index=texts)

print("📊 Bag-of-Words представяне:\n")
print(bow_df.to_string())

print("\n💡 Всяка колона е признак (feature), всеки ред е документ.")
print("   Забележете: губим информация за реда на думите!")

In [ ]:
# Проблем с Bag-of-Words: губим ред
sentence1 = "кучето гони котката"
sentence2 = "котката гони кучето"

vectorizer = CountVectorizer()
X = vectorizer.fit_transform([sentence1, sentence2])

print("❌ Проблем: Bag-of-Words губи реда на думите\n")
print(f"Изречение 1: '{sentence1}'")
print(f"Изречение 2: '{sentence2}'")
print(f"\nВектори: {X.toarray()}")
print(f"Еднакви ли са? {np.array_equal(X[0].toarray(), X[1].toarray())}")

print("\n💡 Различно значение, но СЪЩОТО представяне!")
print("   Това е ограничение на BoW. Ще видим решения в Лекции 2-5.")

### Етикети (Labels)

**Типове етикети в NLP:**

| Задача | Етикети | Пример |
|--------|---------|--------|
| Sentiment analysis | {positive, negative, neutral} | "Страхотен филм!" → positive |
| Spam detection | {spam, not spam} | "БЕЗПЛАТНО!!!" → spam |
| Topic classification | {sports, politics, tech, ...} | "Левски победи ЦСКА" → sports |
| Named Entity Recognition | {B-PER, I-PER, B-ORG, O, ...} | "Иван" → B-PER |

### Разделяне на данните

**Защо разделяме?** Искаме да оценим как моделът работи на НОВИ данни, които не е виждал.

**Три набора:**

| Набор | Цел | Типичен размер |
|-------|-----|----------------|
| **Тренировъчен (Train)** | Обучение на модела | 60-80% |
| **Валидационен (Validation)** | Настройка на хиперпараметри | 10-20% |
| **Тестов (Test)** | Финална оценка | 10-20% |

**Златно правило:** Никога не тренирайте на тестови данни!

In [ ]:
# Зареждане на реален dataset: 20 Newsgroups (ограничени категории)
categories = ['rec.sport.baseball', 'sci.space', 'comp.graphics', 'talk.politics.misc']

print("📥 Зареждане на 20 Newsgroups dataset...\n")

newsgroups = fetch_20newsgroups(
    subset='all',
    categories=categories,
    remove=('headers', 'footers', 'quotes'),
    random_state=42
)

texts = newsgroups.data
labels = newsgroups.target
label_names = newsgroups.target_names

print(f"Брой документи: {len(texts)}")
print(f"Брой категории: {len(label_names)}")
print(f"\nКатегории:")
for i, name in enumerate(label_names):
    count = sum(labels == i)
    print(f"  {i}: {name} ({count} документа)")

In [ ]:
# Показваме примерен документ
sample_idx = 42
sample_text = texts[sample_idx]
sample_label = label_names[labels[sample_idx]]

print(f"📄 Примерен документ (категория: {sample_label})\n")
print("=" * 70)
# Показваме първите 500 символа
print(sample_text[:500] + "..." if len(sample_text) > 500 else sample_text)
print("=" * 70)

In [ ]:
# Разделяне на данните
# Първо: train + temp (test + validation)
X_train, X_temp, y_train, y_temp = train_test_split(
    texts, labels, test_size=0.3, random_state=42, stratify=labels
)

# После: разделяме temp на validation и test
X_val, X_test, y_val, y_test = train_test_split(
    X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
)

print("📊 Разделяне на данните:\n")
print(f"  Тренировъчен набор: {len(X_train)} документа ({len(X_train)/len(texts)*100:.0f}%)")
print(f"  Валидационен набор: {len(X_val)} документа ({len(X_val)/len(texts)*100:.0f}%)")
print(f"  Тестов набор:       {len(X_test)} документа ({len(X_test)/len(texts)*100:.0f}%)")

# Проверка за стратификация
print(f"\n💡 Проверка за стратификация (разпределение на класове):")
print(f"   Train: {np.bincount(y_train)}")
print(f"   Val:   {np.bincount(y_val)}")
print(f"   Test:  {np.bincount(y_test)}")

---
## 4. Парадигми на машинно самообучение

### Три основни парадигми

| Парадигма | Данни | Цел | NLP примери |
|-----------|-------|-----|-------------|
| **Обучение с учител** | X + Y | Предсказване на Y от X | Класификация, NER, превод |
| **Обучение без учител** | Само X | Откриване на структура | Topic modeling, clustering |
| **Обучение с подсилване** | Състояния + награди | Максимизиране на награда | RLHF (Лекция 8) |

### Обучение с учител (Supervised Learning)

**Имаме:** Тренировъчни примери с етикети $(x^{(i)}, y^{(i)})$

**Искаме:** Модел $h(x)$, който предсказва $y$ за нови $x$

**NLP задачи:**
- **Класификация на текст:** Sentiment, spam, topic
- **Sequence labeling:** NER, POS tagging
- **Seq2seq:** Машинен превод, summarization

In [ ]:
# Обучение на прост текстов класификатор
print("🎯 Обучение на текстов класификатор\n")

# Стъпка 1: Векторизация (текст → числа)
vectorizer = TfidfVectorizer(max_features=5000, stop_words='english')
X_train_vec = vectorizer.fit_transform(X_train)
X_val_vec = vectorizer.transform(X_val)
X_test_vec = vectorizer.transform(X_test)

print(f"Стъпка 1: Векторизация")
print(f"  Размерност на входа: {X_train_vec.shape}")
print(f"  (документи x признаци)")

# Стъпка 2: Обучение на модел
model = LogisticRegression(max_iter=1000, random_state=42)
model.fit(X_train_vec, y_train)

print(f"\nСтъпка 2: Обучение на Logistic Regression")
print(f"  ✓ Моделът е обучен!")

# Стъпка 3: Предсказване
y_train_pred = model.predict(X_train_vec)
y_val_pred = model.predict(X_val_vec)

print(f"\nСтъпка 3: Оценка")
print(f"  Train accuracy: {accuracy_score(y_train, y_train_pred):.3f}")
print(f"  Val accuracy:   {accuracy_score(y_val, y_val_pred):.3f}")

In [ ]:
# Показваме няколко предсказания
print("🔮 Примерни предсказания:\n")

for i in range(5):
    text = X_val[i][:100] + "..." if len(X_val[i]) > 100 else X_val[i]
    true_label = label_names[y_val[i]]
    pred_label = label_names[y_val_pred[i]]
    correct = "✓" if y_val[i] == y_val_pred[i] else "✗"
    
    print(f"{correct} Текст: \"{text}\"")
    print(f"  Истински: {true_label}, Предсказан: {pred_label}\n")

### Обучение без учител (Unsupervised Learning)

**Имаме:** Само данни $x^{(i)}$ (без етикети)

**Искаме:** Открием скрита структура в данните

**NLP приложения:**
- **Topic modeling:** Откриване на теми в колекция документи
- **Clustering:** Групиране на сходни документи
- **Word embeddings:** Представяне на думи като вектори (Лекция 2)
- **Language modeling:** Предвиждане на следваща дума (основа на LLM!)

### Обучение с подсилване (Reinforcement Learning)

**Имаме:** Агент, който взима действия и получава награди

**Искаме:** Стратегия, която максимизира общата награда

**В NLP: RLHF (Reinforcement Learning from Human Feedback)**
- Използва се за alignment на LLM (ChatGPT, Claude)
- Човек оценява отговорите на модела
- Моделът се обучава да генерира по-добре оценени отговори

📌 **Ще видим RLHF в детайли в Лекция 8!**

---
## 5. Генерализация: Пренагаждане и ъндърфитване

### Централният проблем в ML

**Цел:** Модел, който работи добре на НОВИ данни (не само на тренировъчните)

**Два начина да се провалим:**

| Проблем | Описание | Симптом |
|---------|----------|--------|
| **Ъндърфитване** | Модел твърде прост | Лоша точност навсякъде |
| **Пренагаждане** | Модел запомня тренировъчни данни | Висока train, ниска test точност |

### Пренагаждане в текстови модели

**Какво означава модел да "запомни" текст?**

- Научава специфични фрази от тренировъчните данни
- Не разпознава перифразирания
- Чувствителен към специфичен речник

**Пример:**
- Train: "Страхотен филм, препоръчвам!" → positive
- Test: "Невероятен филм, гледайте го!" → ??? (нови думи)

In [ ]:
# Демонстрация на пренагаждане с различни размери на речника
print("📈 Демонстрация на пренагаждане\n")

# Взимаме малък subset за по-ясна демонстрация
n_samples = 500
X_train_small = X_train[:n_samples]
y_train_small = y_train[:n_samples]

vocab_sizes = [100, 500, 1000, 2000, 5000, 10000]
train_scores = []
val_scores = []

for vocab_size in vocab_sizes:
    # Векторизация с различен размер на речника
    vec = TfidfVectorizer(max_features=vocab_size, stop_words='english')
    X_tr = vec.fit_transform(X_train_small)
    X_v = vec.transform(X_val)
    
    # Обучение
    clf = LogisticRegression(max_iter=1000, random_state=42)
    clf.fit(X_tr, y_train_small)
    
    # Оценка
    train_acc = accuracy_score(y_train_small, clf.predict(X_tr))
    val_acc = accuracy_score(y_val, clf.predict(X_v))
    
    train_scores.append(train_acc)
    val_scores.append(val_acc)
    
    print(f"Vocab size: {vocab_size:5d} | Train: {train_acc:.3f} | Val: {val_acc:.3f}")

print("\n💡 Забележете: С увеличаване на vocab_size, train accuracy расте,")
print("   но val accuracy на някъде спира да се подобрява или дори пада!")

In [ ]:
# Визуализация на пренагаждане
plt.figure(figsize=(10, 6))
plt.plot(vocab_sizes, train_scores, 'o-', label='Тренировъчен набор', linewidth=2, markersize=8)
plt.plot(vocab_sizes, val_scores, 's-', label='Валидационен набор', linewidth=2, markersize=8)

# Зона на пренагаждане
gap = np.array(train_scores) - np.array(val_scores)
max_gap_idx = np.argmax(gap)
plt.axvline(x=vocab_sizes[max_gap_idx], color='red', linestyle='--', alpha=0.5, 
            label='Максимална разлика (пренагаждане)')

plt.xlabel('Размер на речника (брой признаци)', fontsize=12)
plt.ylabel('Точност (Accuracy)', fontsize=12)
plt.title('Пренагаждане: Train vs Validation Accuracy', fontsize=14)
plt.legend(fontsize=11)
plt.xscale('log')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Интерпретация:")
print("   • Голяма разлика между train и val = пренагаждане")
print("   • Оптималният vocab_size балансира между двете криви")

In [ ]:
# Learning curves: как се променя точността с количеството данни
print("📊 Изчисляване на learning curves...\n")

# Фиксираме vectorizer
vectorizer = TfidfVectorizer(max_features=2000, stop_words='english')
X_all_vec = vectorizer.fit_transform(X_train)

train_sizes, train_scores_lc, val_scores_lc = learning_curve(
    LogisticRegression(max_iter=1000, random_state=42),
    X_all_vec, y_train,
    train_sizes=np.linspace(0.1, 1.0, 10),
    cv=5,
    scoring='accuracy',
    n_jobs=-1
)

# Средни стойности и стандартни отклонения
train_mean = train_scores_lc.mean(axis=1)
train_std = train_scores_lc.std(axis=1)
val_mean = val_scores_lc.mean(axis=1)
val_std = val_scores_lc.std(axis=1)

print("✓ Learning curves изчислени!")

In [ ]:
# Визуализация на learning curves
plt.figure(figsize=(10, 6))

plt.plot(train_sizes, train_mean, 'o-', label='Тренировъчен набор', linewidth=2)
plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, alpha=0.1)

plt.plot(train_sizes, val_mean, 's-', label='Валидационен набор (CV)', linewidth=2)
plt.fill_between(train_sizes, val_mean - val_std, val_mean + val_std, alpha=0.1)

plt.xlabel('Брой тренировъчни примери', fontsize=12)
plt.ylabel('Точност (Accuracy)', fontsize=12)
plt.title('Learning Curves: Как количеството данни влияе на точността', fontsize=14)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

print("💡 Интерпретация:")
print("   • С повече данни валидационната точност се подобрява")
print("   • Разликата между кривите намалява (по-малко пренагаждане)")
print("   • Ако кривите се сближават → добра генерализация")

### Bias-Variance Tradeoff

**Грешката на модела се разделя на:**

$$\text{Error} = \text{Bias}^2 + \text{Variance} + \text{Irreducible Noise}$$

| Компонент | Описание | Резултат от |
|-----------|----------|-------------|
| **Bias** | Систематична грешка | Твърде прост модел (ъндърфитване) |
| **Variance** | Чувствителност към данни | Твърде сложен модел (пренагаждане) |
| **Noise** | Шум в данните | Неизбежен |

**Целта:** Намерете баланса между bias и variance!

---
## 6. Метрики за оценка на NLP модели

### Защо accuracy не е достатъчна?

**Проблем: Дисбалансирани класове**

Пример: Spam detection
- 95% от имейлите са не-spam
- Модел, който винаги казва "не-spam" има 95% accuracy!
- Но пропуска всеки spam...

In [ ]:
# Демонстрация на проблема с accuracy при дисбаланс
np.random.seed(42)

# Симулираме дисбалансирани данни: 95% not-spam, 5% spam
n_total = 1000
n_spam = 50
n_not_spam = n_total - n_spam

y_true = np.array([1] * n_spam + [0] * n_not_spam)  # 1 = spam, 0 = not spam

# "Тъп" модел: винаги предсказва not-spam
y_pred_naive = np.zeros(n_total)

print("🎯 Проблем с accuracy при дисбалансирани данни\n")
print(f"Разпределение: {n_spam} spam ({n_spam/n_total*100:.0f}%), {n_not_spam} not-spam ({n_not_spam/n_total*100:.0f}%)")
print(f"\n'Тъп' модел (винаги казва 'not-spam'):")
print(f"  Accuracy: {accuracy_score(y_true, y_pred_naive):.1%}")
print(f"  Но: открити spam съобщения = 0 от {n_spam}!")

print("\n💡 Accuracy може да е подвеждаща при дисбалансирани класове!")

### Precision, Recall, F1-score

**Confusion Matrix:**

|  | Predicted Positive | Predicted Negative |
|--|-------------------|-------------------|
| **Actual Positive** | True Positive (TP) | False Negative (FN) |
| **Actual Negative** | False Positive (FP) | True Negative (TN) |

**Метрики:**

$$\text{Precision} = \frac{TP}{TP + FP}$$
"От всички, които предсказахме като positive, колко са истински positive?"

$$\text{Recall} = \frac{TP}{TP + FN}$$
"От всички истински positive, колко открихме?"

$$F_1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$
Хармонична средна на precision и recall

In [ ]:
# Оценка на нашия класификатор с всички метрики
y_test_pred = model.predict(X_test_vec)

print("📊 Пълна оценка на класификатора\n")
print("=" * 70)
print(classification_report(y_test, y_test_pred, target_names=label_names))
print("=" * 70)

In [ ]:
# Визуализация на confusion matrix
cm = confusion_matrix(y_test, y_test_pred)

plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
            xticklabels=[n.split('.')[-1] for n in label_names],
            yticklabels=[n.split('.')[-1] for n in label_names])
plt.xlabel('Предсказан клас', fontsize=12)
plt.ylabel('Истински клас', fontsize=12)
plt.title('Confusion Matrix', fontsize=14)
plt.tight_layout()
plt.show()

print("💡 Интерпретация:")
print("   • Диагоналът показва правилни класификации")
print("   • Извън диагонала са грешките")
print("   • Можем да видим кои класове се бъркат най-често")

### Кога коя метрика?

| Ситуация | Важна метрика | Пример |
|----------|---------------|--------|
| Скъпи False Positives | **Precision** | Spam в inbox е досадно |
| Скъпи False Negatives | **Recall** | Пропускане на болест е опасно |
| Балансиран компромис | **F1-score** | Повечето случаи |
| Балансирани класове | **Accuracy** | Когато класовете са равномерни |

### Специфични метрики за NLP (preview)

**За генериране на текст:**
- **BLEU:** Сравнява генериран текст с референтен (машинен превод)
- **ROUGE:** За summarization
- **Perplexity:** Колко "изненадан" е езиковият модел (Лекция 2)

**За sequence labeling:**
- **Token-level accuracy:** За всеки token поотделно
- **Entity-level F1:** За NER - цялостни entities

📌 **Ще видим perplexity в детайли в следващата лекция!**

---
## 7. NLP-специфични предизвикателства (Preview)

### Проблеми, които ще решаваме в следващите лекции

**1. Как представяме текст като числа?** (Лекция 2)
- BoW губи ред и семантика
- Word embeddings: думи като вектори

**2. Какво е "дума"?** (Лекция 3)
- "don't" → една или две думи?
- "New York" → една или две?
- Tokenization и subwords

**3. Как обработваме поредици?** (Лекция 4-5)
- Изреченията имат различна дължина
- Recurrent Neural Networks → Transformers

**4. Как се справяме с контекста?** (Лекция 5)
- "банка" = финансова или речна?
- Attention механизъм

In [ ]:
# Демонстрация: Out-of-Vocabulary (OOV) проблем
train_texts = ["Обичам машинно обучение", "ML е интересно"]
test_texts = ["Deep learning е страхотно"]  # "Deep" и "learning" не са в train

vec = CountVectorizer()
vec.fit(train_texts)

print("❓ Out-of-Vocabulary (OOV) проблем\n")
print(f"Тренировъчен речник: {vec.get_feature_names_out()}")
print(f"\nТест текст: '{test_texts[0]}'")
print(f"Вектор: {vec.transform(test_texts).toarray()}")

print("\n💡 'Deep' и 'learning' не са в речника → игнорирани!")
print("   Това е сериозен проблем за BoW.")
print("   Решение: Subword tokenization (Лекция 3)")

---
## 8. Практически NLP Workflow

### Стъпки за решаване на NLP задача

```
1. Дефинирай задачата
   ↓
2. Събери и разгледай данните
   ↓
3. Предобработка (preprocessing)
   ↓
4. Извличане на признаци / Tokenization
   ↓
5. Раздели данните (train/val/test)
   ↓
6. Избери и обучи модел
   ↓
7. Оцени на validation set
   ↓
8. Tune hyperparameters
   ↓
9. Финална оценка на test set
   ↓
10. Error analysis
```

### Чести грешки в NLP

**1. Data leakage (изтичане на данни)**
- Fit vectorizer на test данни
- Използване на бъдеща информация при time-series

**2. Test set contamination**
- Особено важно за LLM: моделът може да е виждал test данните при pretraining
- "Benchmark hacking"

**3. Vocabulary mismatch**
- Train на един домейн, test на друг
- OOV думи

**4. Игнориране на preprocessing**
- Lowercasing, punctuation, special characters
- Различни preprocessing за train/test

In [ ]:
# Error analysis: какви грешки прави моделът?
print("🔍 Error Analysis: Къде греши моделът?\n")
print("=" * 70)

# Намираме грешни предсказания
errors = np.where(y_test != y_test_pred)[0]
print(f"Общо грешки: {len(errors)} от {len(y_test)} ({len(errors)/len(y_test)*100:.1f}%)\n")

# Показваме няколко примера
print("Примерни грешки:\n")
for i, idx in enumerate(errors[:5]):
    text = X_test[idx][:150] + "..." if len(X_test[idx]) > 150 else X_test[idx]
    true_label = label_names[y_test[idx]].split('.')[-1]
    pred_label = label_names[y_test_pred[idx]].split('.')[-1]
    
    print(f"Пример {i+1}:")
    print(f"  Текст: \"{text}\"")
    print(f"  Истински: {true_label}")
    print(f"  Предсказан: {pred_label}\n")

print("💡 Error analysis помага да разберем слабостите на модела!")

---
## 9. Обобщение и мост към Лекция 2

### Ключови изводи от днес

**1. ML е учене от данни**
- Task, Experience, Performance metric
- Формална нотация: $h: \mathcal{X} \to \mathcal{Y}$

**2. Текстът изисква специално представяне**
- Bag-of-Words: просто, но губи информация
- TF-IDF: претегля важността на думите

**3. Три парадигми на обучение**
- С учител: имаме етикети
- Без учител: търсим структура
- С подсилване: награди за действия

**4. Генерализацията е ключова**
- Пренагаждане vs ъндърфитване
- Train/val/test split

**5. Метриките зависят от задачата**
- Accuracy, Precision, Recall, F1
- Confusion matrix за диагностика

### Следваща лекция: Езикови модели и представяне на думи

**Ще отговорим на:**

- Как да представим думи, така че "крал" и "кралица" да са близки?
- Какво е езиков модел?
- Как работи Word2Vec?
- Какво е perplexity?

**Защо е важно:**
- Word embeddings са основа на съвременните LLM
- Езиковите модели са в сърцето на ChatGPT, Claude, GPT-4

---
## Ресурси

### Препоръчително четене

**Статии:**
1. "A Few Useful Things to Know About Machine Learning" - Pedro Domingos (2012)
2. "Natural Language Processing (almost) from Scratch" - Collobert et al. (2011)

**Учебници:**
1. "Speech and Language Processing" - Jurafsky & Martin (3rd ed.) - Chapter 4
2. "Introduction to Information Retrieval" - Manning et al. - Chapter 13
3. "Pattern Recognition and Machine Learning" - Bishop - Chapter 1

**Online:**
1. Stanford CS229 Lecture Notes
2. Stanford CS224N Lecture 1
3. Scikit-learn documentation

### Упражнения за вкъщи

**Упражнение 1: Spam класификатор**
- Заредете SMS Spam Collection dataset
- Разделете на train/val/test
- Обучете класификатор (BoW + Logistic Regression)
- Оценете с precision, recall, F1
- Анализирайте грешките

**Упражнение 2: Сравнение на признаци**
- Сравнете unigrams vs bigrams vs trigrams
- Начертайте learning curves за всеки
- Кой представя данните най-добре?

**Упражнение 3: Hyperparameter tuning**
- Експериментирайте с различни стойности на:
  - `max_features` в TfidfVectorizer
  - `C` в LogisticRegression
- Използвайте validation set за избор
- Оценете финално на test set

---
## Край на Лекция 1

### Благодаря за вниманието!

**Въпроси?**

---

**Следваща лекция:** Езикови модели и представяне на думи